Look for any .tar files

In [17]:
ls *.tar

20200508_02_seattle_fullcity_processed.tar*  run07.tar  run16.tar  run25.tar
20200510_03_seattle_fullcity_processed.tar*  run08.tar  run17.tar  run26.tar
run01_output.tar*                            run09.tar  run18.tar  run27.tar
run01.tar                                    run10.tar  run19.tar  run28.tar
run02.tar                                    run11.tar  run20.tar  run29.tar
run03.tar                                    run12.tar  run21.tar  run30.tar
run04.tar                                    run13.tar  run22.tar  run31.tar
run05.tar                                    run14.tar  run23.tar  run32.tar
run06.tar                                    run15.tar  run24.tar  run33.tar


Define the .tar file we want to unpack and postprocess

In [18]:
import os

tarFile = 'run33.tar'
runWNum = tarFile.split('.')[0]
runNum = runWNum[-2:]
runName = next(filter(lambda x:len(x.split('_'))>1 and x.split('_')[1]==runNum,os.listdir('../../Processed_Data/Pulsar/')))

Unpack .tar file and rename folder

In [19]:
%%bash -s $tarFile $runWNum $runName
tar -xf $1
mv $2 $3

Do postprocessing on the unpacked results

In [20]:
!pip install pickle5 --user

In [21]:
%%bash -s $runName
pip install pygeos 
pip install geopandas
pip install sqlalchemy_utils
python postProcessing.py $1

/home/jupyter/projects/PRJ-2759/Computer_Vision/CorrectedImages/postProcessing.py:67: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  df = jobPickleToDataframe(pkldata,(runNum,date,subfolder))
/home/jupyter/projects/PRJ-2759/Computer_Vision/CorrectedImages/postProcessing.py:200: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2926
Right CRS: PROJCS["NAD83(HARN) / Washington North (ftUS)",GEO ...

  merged_file = gpd.sjoin(gdf, census_tracts, how='left', op='within')
/home/jupyter/projects/PRJ-2759/Computer_Vision/CorrectedImages/postProcessing.py:67: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  df = jobPickleToDataframe(pkldata,(runNum,date,subfolder))
/home/jup

Loading output4.pkl...
Loading output15.pkl...
Loading output1.pkl...
Loading output5.pkl...
Loading output14.pkl...
Loading output8.pkl...
Loading output22.pkl...
Loading output24.pkl...
Loading output6.pkl...
Loading output20.pkl...
Loading output2.pkl...
Loading output16.pkl...
Loading output23.pkl...
Loading output10.pkl...
Loading output18.pkl...
Loading output9.pkl...
Loading output3.pkl...
Loading output21.pkl...
Loading output11.pkl...
Loading output7.pkl...
Loading output19.pkl...
Loading output17.pkl...
Loading output12.pkl...
Loading output13.pkl...


%%bash -s $runName
#pip install pygeos 
#pip install geopandas
#pip install sqlalchemy_utils
python postProcessing_run14.py $1

Crop out the pedestrians from corrected images

import os
croppedFolder = 'CroppedImages'
if croppedFolder not in os.listdir():
    os.mkdir(croppedFolder)

runFolder = 'run{}'.format(runNum)
runDir = os.path.join(croppedFolder,runFolder)
dfDir = os.path.join('DetectionDfs','Run{}_Detections.pkl'.format(runNum))

if runFolder not in os.listdir(croppedFolder):
    os.mkdir(runDir)

%%bash -s $runName $runDir $dfDir
python cropImages.py $1 $2 $3

import cropImages

from PIL import Image
imgList = cropImages.listImages(runName)
df = pd.read_pickle(dfDir)
detectionDf = df
outDir = runDir

for i,imgFname in enumerate(imgList):

        # Load image and parse data info needed to get relevant detections
        img = Image.open(imgFname)
        folder,num = imgFname.split('/')[1:]

        if folder not in os.listdir(outDir):
            os.mkdir(os.path.join(outDir,folder))

        num,side = num.split('_')
        side = side.split('.')[0]

        detects = detectionDf[(detectionDf.Subfolder==folder)&\
                              (detectionDf.Filename==num[-4:])&\
                              (detectionDf.Side==side)]
        '''
        if len(detects) > 0:
            print('{} detections in image {}'.format(len(detects),i))
        '''
        
        # Loop over detections and crop each out of relevant image
        for j,row in detects.iterrows():

            detId,l,t,r,b = row[['DetectionID','Xmin','Ymin','Xmax','Ymax']]
            l,t,r,b = int(l),int(t),int(r),int(b)
            croppedImg = img.crop((l,t,r,b))

            outName = '{}_{}_{}.png'.format(num,side,detId)
            try:
                croppedImg.save(os.path.join(outDir,folder,outName))
            except Exception as e:
                print('Image size: {}'.format(img.size))
                print('bbox: {}'.format((l,t,r,b)))
                raise e

        if i%100 == 0:
            print('Done with image {}!'.format(i))
            # print('Output dir: {}'.format(os.path.join(outDir,folder,outName)))

Concatenate all of the pairs dataframes

In [22]:
pairFoldName = 'run{}_pairs'.format(runNum)
oldFoldName = 'run{}_pairs'.format(int(runNum))

In [23]:
%%bash -s $oldFoldName $pairFoldName
#mv $1 $2

In [24]:

import pandas as pd
pairDfList = []
for fname in os.listdir('run{}_pairs'.format(runNum)):
    df = pd.read_pickle(os.path.join('run{}_pairs'.format(runNum),fname))
    pairDfList.append(df)
    
pairDf = pd.concat(pairDfList)
pairDf.to_pickle('run{}_pairs/pairs.pkl'.format(runNum))

FileNotFoundError: [Errno 2] No such file or directory: 'run33_pairs'